# Election Simulation 

(Original version Created by Karl Sigman)  


Using the 51 [Electoral College](https://www.archives.gov/electoral-college/2000) numbers that were used in the 2000 USA Presidential Election, estimate the number of ways there could have been a tie in the 2000 Presidential Election.  Use Monte Carlo simulation (using *iid* Bernoulli (1/2); e.g., a fair coin flip for each state) to simulate many elections. The fraction of elections that end in tie multiplied by the number of possible outcomes is your estimate. The exact answer to this question was determined in 2009 by K. Sigman and O. Watanabe to be 17,150,271,124,366.


Repeat using the [EC numbers](https://www.archives.gov/electoral-college/2020) from the 2020 election. That is create a new list of EC values and feed it to your function.  *NOTE: Maine and Nebraska do not use a winner-take-all electoral college system. In this project I make the simplifying assumtion that they do.*



In [4]:
import numpy as np

v_2000 = [9,3,8,6,54,8,8,3,3,25,13,4,4,22,12,7,6,8,9,4,10,12,18,10,7,11,
          3,5,4,4,15,5,33,14,3,21,8,7,23,4,8,3,11,32,5,3,13,11,5,11,3]

v_2020 = [9,3,11,6,55,9,7,3,3,29,16,4,4,20,11,6,6,8,8,4,10,11,16,10,6,10,
          3,5,6,4,14,5,29,15,3,18,7,7,20,4,9,3,11,38,6,3,13,12,5,10,3]

# ec: this is the list of EC values to be used in the simulation
# target: this is the number of EC values candidate 1 should win
# trials: this is the number of trials the MC simulation should use

def target_estimator(ec, target,trials):
    # generate an array of number of trials by number of electoral votes
    random_numbers = np.random.randint(2, size=(trials, len(ec)))
    
    total_outcomes = np.dot(random_numbers, ec)
    num_of_ties = [outcome for outcome in total_outcomes if outcome == target]

    # estimation of N_t
    tgt_est = (len(num_of_ties) * (2**(len(ec)))) / trials
    return (int(tgt_est))
    
 
# now use the function to determine the number of ways a tie can occur

result_2000 = target_estimator(v_2000,269,1000000)
result_2020 = target_estimator(v_2020,269,1000000)

print('result 2000: ', f'{result_2000:,}')
print('result 2020: ', f'{result_2020:,}')

result 2000:  17,377,139,162,209
result 2020:  16,953,800,797,236


In the year 2000 (Bush versus Gore), the situation right before the election was this: Bush had (in his pocket) 24 states totaling 210 EC votes, while Gore had 10 states totaling 146 EC votes. There were 17 states left over, the “Battleground States”, in which it was supposedly unclear who would win them. Look at the file 2000.pdf (Located in Data folder) to see exactly what states made up the 17, and the EC numbers for them.

Assuming each Battleground State outcome is determined by an *iid* fair coin toss; Bernoulli (1/2). A simulation (using 1 million copies to average, using Monte Carlo) can be created to obtain the probability that Bush would win the election, and the probability that Gore would win the election, and the probability of a tie.


In [5]:
import numpy as np

tie = 269

v_in_play= [6,3,25,22,7,4,18,10,11,4,4,7,23,11,11,5,11]

# this function returns an estimate for the probability that candidate 1
# wins in a US Presidential election given that they already have
# ec_in_the_bag EC votes.
# v_left is a list of the remaining EC numbers,
# trials is the number of trials to be used in the MC simulation.
# This function assumes that the probability of winning any remaining state
# is 0.5

def ec_estimator(ec_in_the_bag, v_left,trials):
    random_numbers = np.random.randint(2, size=(trials, len(v_left)))

    battle_state_outcomes = np.dot(random_numbers, v_left)
    total_vote_left = sum(v_left)

    win_outcomes = [outcome for outcome in battle_state_outcomes if (
        ec_in_the_bag + outcome) > tie]

    win_prob = len(win_outcomes) / trials
    return (win_prob)
    

# estimate for Bush win
print('Bush win: ',ec_estimator(210,v_in_play,1000000))

# estimate for Gore win
print('Gore win: ',ec_estimator(146,v_in_play,1000000))

#estimate for Tie
print('tie: ',target_estimator(v_in_play,269-146,1000000)/2**len(v_in_play))

#check that it's the same (close) with
print('tie: ',target_estimator(v_in_play,269-210,1000000)/2**len(v_in_play))



Bush win:  0.879405
Gore win:  0.112192
tie:  0.007598876953125
tie:  0.0075531005859375


In the 2000.pdf file, there are also the probabilities that had been determined by extensive polling for Gore winning each of the 17 states. The following function redos the previous simulation using the 17 Bernoulli $(p_i)$. The idea now is that each of the 17 states has its own coin so to speak.

In [6]:
import numpy as np

v_in_play= [6,3,25,22,7,4,18,10,11,4,4,7,23,11,11,5,11]

gore_poll_prob = [0.395, 0.143, 0.893, 0.999, 0.42, 0.5, 0.997, 0.999,
                 0.236, 0.146, 0.731, 0.602, 0.989, 0.289, 0.753, 0.302, 0.946]

def ec_estimator_with_poll(ec_in_the_bag, v_left, v_poll, trials):
    #toss the first entry in v_poll
    random_toss = np.random.choice((0,1), size=(trials,1), p = [1-v_poll[0], v_poll[0]])

    #toss the remaining in the v_poll
    for i in range(1,len(v_poll)):
        arr = np.random.choice((0,1), size=(trials,1), p = [1-v_poll[i], v_poll[i]])
        # concatenate remaining tosses to random_toss
        random_toss=np.concatenate((np.copy(random_toss), np.copy(arr)), axis=1)

    battle_state_outcomes = np.dot(random_toss, v_left)

    win_outcomes = [outcome for outcome in battle_state_outcomes if (
        ec_in_the_bag + outcome) > tie]

    win_prob = len(win_outcomes) / trials
    return (win_prob)


bush_poll_prob = [ (1 - x) for x in gore_poll_prob ]

bush_win = ec_estimator_with_poll(210, v_in_play,bush_poll_prob, 1000000)
gore_win = ec_estimator_with_poll(146,v_in_play, gore_poll_prob, 1000000)

# estimate for Bush win with poll
print('Bush win with poll: ', ec_estimator_with_poll(210, v_in_play,bush_poll_prob, 1000000))

# estimate for Gore win with poll
print('Gore win with poll: ', ec_estimator_with_poll(146,v_in_play, gore_poll_prob, 1000000))

# estimate for a tie
print('tie: ', (1 - bush_win - gore_win))


Bush win with poll:  0.142487
Gore win with poll:  0.847734
tie:  0.010622999999999938
